In [1]:
import pandas as pd
df = pd.read_csv('/Users/pujaarajan/Documents/GitHub/nlp_fib/data/raw/enron_05_17_2015_with_labels_v2_100K_chunk_1_of_6.csv', usecols= [*range(2, 6),13], dtype={13:str})

# Pytorch Bidirectional Language Model
## https://gist.github.com/tokestermw/912042a85a1d53169c2dc7253dca55f6#file-birnnlm_pytorch-py

In [1]:
import torch, torch.nn as nn
from torch.autograd import Variable

In [2]:
text = ['BOS', 'How', 'are', 'you', 'EOS']
seq_len = len(text)
batch_size = 1
embedding_size = 1
hidden_size = 1
output_size = 1

In [3]:
random_input = Variable(
    torch.FloatTensor(seq_len, batch_size, embedding_size).normal_(), requires_grad=False)

print(random_input)



tensor([[[ 0.0952]],

        [[ 0.4962]],

        [[ 0.4910]],

        [[-1.0986]],

        [[ 0.3712]]])
RNN(1, 1, bidirectional=True)
Linear(in_features=2, out_features=1, bias=True)
tensor([[[ 0.4962]],

        [[ 0.4910]],

        [[-1.0986]]])


In [4]:
bi_rnn = torch.nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)

print (bi_rnn)

bi_output, bi_hidden = bi_rnn(random_input)

# stagger
forward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]
staggered_output = torch.cat((forward_output, backward_output), dim=-1)

linear = nn.Linear(hidden_size * 2, output_size)

print(linear)

# only predict on words
labels = random_input[1:-1]

print (labels)

# for language models, use cross-entropy :)
loss = nn.MSELoss()
output = loss(linear(staggered_output), labels)

RNN(1, 1, bidirectional=True)
Linear(in_features=2, out_features=1, bias=True)
tensor([[[ 0.4962]],

        [[ 0.4910]],

        [[-1.0986]]])


# Pytorch Bidirectional RNN

In [47]:
import torch, torch.nn as nn
from torch.autograd import Variable

text = ['BOS', 'How', 'are', 'you','EOS', 'BOS', 'Hi', 'Bye' 'EOS']

seq_len = len(text)
batch_size = 1 # Change this
embedding_size = 1 # Could be the bumber of words you start with to predict next or 
hidden_size = 1
output_size = 1

#
     torch.FloatTensor(seq_len, batch_size, embedding_size).normal_(), requires_grad=False)

random_input

tensor([[[ 1.8969]],

        [[-1.6470]],

        [[-1.6635]],

        [[-0.6729]],

        [[-0.2843]],

        [[ 0.5671]],

        [[-0.0598]],

        [[-0.6298]]])

In [48]:
# Pretrained model
bi_rnn = torch.nn.RNN(
    input_size=embedding_size, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)

bi_output, bi_hidden = bi_rnn(random_input)

print(bi_output)

print(bi_hidden)

tensor([[[-0.0742, -0.2596]],

        [[-0.8175, -0.4755]],

        [[-0.7459, -0.4198]],

        [[-0.5891, -0.3133]],

        [[-0.5346, -0.2494]],

        [[-0.3344, -0.1926]],

        [[-0.5318, -0.2071]],

        [[-0.6156, -0.1532]]], grad_fn=<CatBackward>)
tensor([[[-0.6156]],

        [[-0.2596]]], grad_fn=<StackBackward>)


In [34]:
# stagger
forward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]

print(forward_output)
print(backward_output)

#forward_output - first 3 word features BOS How Are
#backward_output has the last word features Are You EOS

tensor([[[-0.9471]],

        [[-0.8963]],

        [[-0.0850]]], grad_fn=<SliceBackward>)
tensor([[[-0.7963]],

        [[-0.9743]],

        [[-0.9580]]], grad_fn=<SliceBackward>)


In [35]:
staggered_output = torch.cat((forward_output, backward_output), dim=-1)

staggered_output 
#first column forward output and second column is backward output
#first column - BOS how are
#second column - are you EOS

tensor([[[-0.9471, -0.7963]],

        [[-0.8963, -0.9743]],

        [[-0.0850, -0.9580]]], grad_fn=<CatBackward>)

In [36]:
linear = nn.Linear(hidden_size * 2, output_size)

linear

Linear(in_features=2, out_features=1, bias=True)

In [40]:
# only predict on words
original_words = random_input[1:-1]

In [41]:
original_words

tensor([[[-0.0421]],

        [[-1.4417]],

        [[ 0.0216]]])

In [43]:
# for language models, use cross-entropy :)
loss = nn.MSELoss()
loss

MSELoss()

In [45]:
linear(staggered_output)

tensor([[[-1.1693]],

        [[-1.2597]],

        [[-1.1365]]], grad_fn=<AddBackward0>)

In [46]:
output = loss(linear(staggered_output), original_words)
output
# calculate mean absolute error bw each element in input x and output y

tensor(0.8816, grad_fn=<MseLossBackward>)

# Pytorch RNN

In [12]:
class LSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, batch_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.hidden2label = nn.Linear(hidden_dim * 2, label_size) # Ä: hidden_dim * 2 for bidirectional
        self.hidden = self.init_hidden()

    def init_hidden(self):

        h0 = Variable(torch.zeros(1 * 2, self.batch_size, self.hidden_dim))
        c0 = Variable(torch.zeros(1 * 2, self.batch_size, self.hidden_dim))
        return h0, c0

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        x = embeds.view(len(sentence), self.batch_size, -1)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y = self.hidden2label(lstm_out[-1])
        return y